# 0) Web Scraping - US Hit Streaming Songs Project

__The following Notebook is the precursor to a four-part series. This Notebook, specifically, collects data from HITS Daily Double's [Streaming Songs Chart](https://hitsdailydouble.com/streaming_songs) and adds it to [this](https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0) Google Sheet__. The steps taken are the following:
- Webscraping the Data
- Importing the Google Sheet and turning it into a dataframe
- Merging data
- Editing the Google Sheet dataframe
- Checking the newly edited Google Sheet dataframe
- Saving final dataframe to Google Sheet

__Overview of the process__: Once I completed parts 1-4 of this project, I decided to go back and improve how I collected data every week by __webscraping weekly streaming data instead of manually inputing it onto Google Sheets (as I did for these last 5 years before I knew how to code)__. In order to do so, this project utilizes [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/), a Python library that pulls data out of HTML and XML files. Once Beautiful Soup is imported, we create two dataframes, one taken straight from HITS Daily Double's chart and the other one being a cleaned-up version of the first dataframe. Then, we import our [Google Sheet](https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0) and join it with the newly-cleaned-up dataframe. Finally, we save the resulting dataframe to the Google Sheet. 

In [1]:
hits_revenue_url = 'https://hitsdailydouble.com/song_revenue_chart'
hits_streaming_url = 'https://hitsdailydouble.com/streaming_songs'

In [2]:
import requests
from lxml import html
import numpy as np
import pandas as pd

# Set the display option to not use scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Replace 'your_link_here' with the actual link
# url = 'https://web.archive.org/web/20180325113915/https://hitsdailydouble.com/song_revenue_chart'
response = requests.get(hits_revenue_url)
tree = html.fromstring(response.content)

# XPath for 'Artist | Track | Label'
artist_track_label_xpath = "//td[@class='hits_upcoming_releases_artist']"

# Extract data using XPath
artist_track_label_elements = tree.xpath(artist_track_label_xpath)

# Lists to store data
artists = []
tracks = []
labels = []
lw_values = []
tw_values = []
total_revenue_values = []  # 'Total Revenue' column
sales_revenue_values = []  # 'Sales Revenue' column
sales_values = []  # 'Sales' column
streams_revenue_values = []  # 'Streams Revenue' column
streams_values = []  # 'Streams' column

# Iterate through each element
for element in artist_track_label_elements:
    # Extract text from the strong tag
    strong_text = element.xpath('strong/text()')[0].strip() if element.xpath('strong/text()') else ''
    
    # Split the text on '|'
    parts = strong_text.split('|')
    
    # Extract artist and track
    artist = parts[0].strip() if len(parts) > 0 else ''
    track = parts[1].strip() if len(parts) > 1 else ''
    
    # Extract label information directly from the text content after <br>
    label_elements = element.xpath(".//br/following-sibling::text()")
    label = label_elements[0].strip() if label_elements else ''
    
    # Extract LW value
    lw_element = element.xpath("preceding::td[2]/text()")  # XPath for LW value
    lw = lw_element[0].strip() if lw_element else 'N/A'
    
    # Extract TW value
    tw_element = element.xpath("preceding::td[1]/text()")  # XPath for TW value
    tw = tw_element[0].strip() if tw_element else 'N/A'
    
    # Extract Total Revenue value
    total_revenue_element = element.xpath("following::td[4]/text()")  # XPath for Total Revenue value
    total_revenue = total_revenue_element[0].strip() if total_revenue_element else 'N/A'
    
    # Extract Sales Revenue value from the specific <td> element
    sales_revenue_element = element.xpath("following::td[@class='hits_upcoming_releases_prev_date2']/text()")  
    sales_revenue = sales_revenue_element[0].strip() if sales_revenue_element else 'N/A'
    
    # Extract Sales value from the specific <td> element
    sales_element = element.xpath("following::td[@class='hits_upcoming_releases_prev_date2']/br/following-sibling::text()")
    sales = sales_element[0].strip() if sales_element else 'N/A'
    
    # Extract Streams Revenue value from the specific <td> element
    streams_revenue_element = element.xpath("following::td[@class='hits_upcoming_releases_first_week2']/text()")
    streams_revenue = streams_revenue_element[0].strip() if streams_revenue_element else 'N/A'
    
    # Extract Streams value from the specific <td> element
    streams_element = element.xpath("following::td[@class='hits_upcoming_releases_first_week2']/br/following-sibling::text()")
    streams = streams_element[0].strip() if streams_element else 'N/A'
    
    # Append to lists
    artists.append(artist)
    tracks.append(track)
    labels.append(label)
    lw_values.append(lw)
    tw_values.append(tw)
    total_revenue_values.append(total_revenue)
    sales_revenue_values.append(sales_revenue)
    sales_values.append(sales)
    streams_revenue_values.append(streams_revenue)
    streams_values.append(streams)

# Create DataFrame
revenue_df = pd.DataFrame({
    'LW': lw_values,
    'TW': tw_values,
    'Artist': artists,
    'Track': tracks,
    'Label': labels,
    'Total Revenue': total_revenue_values,
    'Sales Revenue': sales_revenue_values,
    'Sales': sales_values,
    'Streams Revenue': streams_revenue_values,
    'Streams': streams_values
})

# Display the DataFrame
revenue_df



,LW,TW,Artist,Track,Label,Total Revenue,Sales Revenue,Sales,Streams Revenue,Streams
0,1,1,SHABOOZEY,A BAR SONG (TIPSY),EMPIRE (EMPIRE),"$129,165","$12,740","14,000","$129,165","28,388,000"
1,2,2,KENDRICK LAMAR,NOT LIKE US,PGLANG/INTERSCOPE (UMG),"$124,242","$3,731","4,100","$124,242","27,306,000"
2,3,3,POST MALONE FEAT. MORGAN WALLEN,I HAD SOME HELP,BIG LOUD/MERCURY/REPUBLIC (UMG),"$121,467","$6,188","6,800","$121,467","26,696,000"
3,4,4,JIMIN,WHO,BIGHIT/GEFFEN (UMG),"$80,799","$43,862","48,200","$80,799","17,758,000"
4,9,5,BILLIE EILISH,BIRDS OF A FEATHER,DARKROOM/INTERSCOPE (UMG),"$93,261","$2,912","3,200","$93,261","20,497,000"
5,7,6,CHAPPELL ROAN,"GOOD LUCK, BABE!",ISLAND (UMG),"$92,183","$2,639","2,900","$92,183","20,260,000"
6,5,7,SABRINA CARPENTER,PLEASE PLEASE PLEASE,ISLAND (UMG),"$88,602","$2,184","2,400","$88,602","19,473,000"
7,8,8,SABRINA CARPENTER,ESPRESSO,ISLAND (UMG),"$86,878","$3,367","3,700","$86,878","19,094,000"
8,6,9,TOMMY RICHMAN,MILLION DOLLAR BABY,ISO SUPREMACY/PULSE (STEM),"$86,864","$2,548","2,800","$86,864","19,091,000"
9,10,10,MORGAN WALLEN,LIES LIES LIES,BIG LOUD/MERCURY/REPUBLIC (UMG),"$76,704","$2,639","2,900","$76,704","16,858,000"


In [3]:
import numpy as np

# Identify non-numeric values in the 'Streams' column
non_numeric_streams_mask = ~revenue_df['Streams'].str.replace(',', '').str.isnumeric()

# Convert 'Streams' column to numeric, replacing non-numeric values with NaN
revenue_df['Streams'] = pd.to_numeric(revenue_df['Streams'].str.replace(',', ''), errors='coerce')

# Generate random numbers between -500 and 500
random_numbers = np.random.randint(-500, 501, size=len(revenue_df))

# Add random numbers to the 'Streams' column
revenue_df['Streams_editted'] = revenue_df['Streams'] + random_numbers
revenue_df[['TW','Artist', 'Track','Sales','Streams','Streams_editted']]

,TW,Artist,Track,Sales,Streams,Streams_editted
0,1,SHABOOZEY,A BAR SONG (TIPSY),"14,000",28388000,28388154
1,2,KENDRICK LAMAR,NOT LIKE US,"4,100",27306000,27306427
2,3,POST MALONE FEAT. MORGAN WALLEN,I HAD SOME HELP,"6,800",26696000,26695852
3,4,JIMIN,WHO,"48,200",17758000,17757518
4,5,BILLIE EILISH,BIRDS OF A FEATHER,"3,200",20497000,20497308
5,6,CHAPPELL ROAN,"GOOD LUCK, BABE!","2,900",20260000,20260151
6,7,SABRINA CARPENTER,PLEASE PLEASE PLEASE,"2,400",19473000,19473334
7,8,SABRINA CARPENTER,ESPRESSO,"3,700",19094000,19094419
8,9,TOMMY RICHMAN,MILLION DOLLAR BABY,"2,800",19091000,19090725
9,10,MORGAN WALLEN,LIES LIES LIES,"2,900",16858000,16858205


In [4]:
# import numpy as np

# # Identify non-numeric values in the 'Streams' column
# non_numeric_streams_mask = ~revenue_df['Streams Revenue'].str.replace(',', '').str.isnumeric()

# # Convert 'Streams' column to numeric, replacing non-numeric values with NaN
# revenue_df['Streams Revenue'] = pd.to_numeric(revenue_df['Streams Revenue'].str.replace(',', ''), errors='coerce')

# # Generate random numbers between -500 and 500
# random_numbers = np.random.randint(-500, 501, size=len(revenue_df))

# # Add random numbers to the 'Streams' column
# revenue_df['Streams_editted'] = revenue_df['Streams Revenue'] #+ random_numbers
# revenue_df[['TW','Artist', 'Track','Sales Revenue','Streams Revenue','Streams_editted']]

In [5]:
# import numpy as np

# # Identify non-numeric values in the 'Streams' column
# non_numeric_streams_mask = ~df['Streams Revenue'].str.replace(',', '').str.isnumeric()

# # Convert 'Streams' column to numeric, replacing non-numeric values with NaN
# df['Streams Revenue'] = pd.to_numeric(df['Streams Revenue'].str.replace(',', ''), errors='coerce')

# # Generate random numbers between -500 and 500
# random_numbers = np.random.randint(-500, 501, size=len(df))

# # Add random numbers to the 'Streams' column
# df['Streams Revenue'] = df['Streams Revenue'] #+ random_numbers
# df[['TW','Artist', 'Label','Sales Revenue','Streams Revenue']].sort_values('Streams Revenue',ascending=False)

In [6]:
# import numpy as np

# # Identify non-numeric values in the 'Streams' column
# non_numeric_streams_mask = ~df['Streams'].str.replace(',', '').str.isnumeric()

# # Convert 'Streams' column to numeric, replacing non-numeric values with NaN
# df['Streams'] = pd.to_numeric(df['Streams'].str.replace(',', ''), errors='coerce')

# # Generate random numbers between -500 and 500
# random_numbers = np.random.randint(-500, 501, size=len(df))

# # Add random numbers to the 'Streams' column
# df['Streams'] = df['Streams']  + random_numbers
# df[['TW','Artist', 'Label','Sales Revenue','Streams']].sort_values('Streams',ascending=False)

In [7]:
revenue_df[['TW','Artist', 'Track','Sales','Streams','Streams_editted']].sort_values('Streams_editted',ascending=False).to_csv('revenue_chart.csv')

In [8]:
revenue_df[['TW','Artist', 'Label','Sales Revenue','Streams Revenue','Streams_editted']].sort_values('Streams_editted',ascending=False).to_csv('revenue_chart.csv')

In [9]:
revenue_df[['TW','Artist', 'Label','Sales Revenue','Streams Revenue','Streams_editted']]

,TW,Artist,Label,Sales Revenue,Streams Revenue,Streams_editted
0,1,SHABOOZEY,EMPIRE (EMPIRE),"$12,740","$129,165",28388154
1,2,KENDRICK LAMAR,PGLANG/INTERSCOPE (UMG),"$3,731","$124,242",27306427
2,3,POST MALONE FEAT. MORGAN WALLEN,BIG LOUD/MERCURY/REPUBLIC (UMG),"$6,188","$121,467",26695852
3,4,JIMIN,BIGHIT/GEFFEN (UMG),"$43,862","$80,799",17757518
4,5,BILLIE EILISH,DARKROOM/INTERSCOPE (UMG),"$2,912","$93,261",20497308
5,6,CHAPPELL ROAN,ISLAND (UMG),"$2,639","$92,183",20260151
6,7,SABRINA CARPENTER,ISLAND (UMG),"$2,184","$88,602",19473334
7,8,SABRINA CARPENTER,ISLAND (UMG),"$3,367","$86,878",19094419
8,9,TOMMY RICHMAN,ISO SUPREMACY/PULSE (STEM),"$2,548","$86,864",19090725
9,10,MORGAN WALLEN,BIG LOUD/MERCURY/REPUBLIC (UMG),"$2,639","$76,704",16858205


In [10]:
# df.sort_values('Streams',ascending=False, inplace=True)
# df[['Artist', 'Track']] = df[['Artist', 'Track']].apply(lambda x: x.str.lower().str.title())
# df['Artist'] = df['Artist'].str.replace(' Feat.', ', ')
# df['Streams'] = df['Streams'].apply(lambda x: '{:,.0f}'.format(x))
# df['TW'] = range(1, 51)
# # df.loc[1,'Artist'] = 'Zach Bryan'
# # df.loc[9,'Artist'] = 'SZA'
# df.set_index('TW',drop=True,inplace=True)


In [11]:
# df

In [12]:
# df[['Artist', 'Track','Streams']].head(10)

In [13]:
# df_melted[df_melted['artist']==]

In [14]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set options
pd.set_option('display.max_colwidth', None) 

In [15]:
#get url and create our "soup"
# hits_streaming_url = "https://web.archive.org/web/20180327021126/https://hitsdailydouble.com/streaming_songs"
page = requests.get(hits_streaming_url)
soup = BeautifulSoup(page.text, 'lxml')

In [16]:
# Collecting data and creating a temporary df
df_temp = pd.DataFrame(columns=['col', 'strong', 'span'])

"""
Sometimes you need to change soup.findAll('table')[3] from 3 to 4 !!!!!!!!!!!!!!
"""
#get information from the website
for row in soup.findAll('table')[3].findAll('tr'): 
    column = row.find_all('td')
    all_strong=row.find('strong')
    all_span=row.find_all('span')
            
    df_temp = df_temp.append({'col':column, 'strong':all_strong, 
                'span':all_span},ignore_index=True)
df_temp

,col,strong,span
0,"[[LW], [TW], [Artist | Title | Label], [Streams], [Change ]]",None,[]
1,"[[\n2 ], [1 ], [[<tr><td><img class=""hits_album_chart_featured_image"" src=""https://ed.hitsdailydouble.com/inc/hits/sales/KENDRICK LAMAR - NOT LIKE US.png""/></td><td class=""hits_album_chart_item_top_full_details""><span class=""hits_album_chart_item_top_full_details_artist"">KENDRICK LAMAR | NOT LIKE US</span><br/><span class=""hits_album_chart_item_top_dull_details_release"">PGLANG/INTERSCOPE (UMG)</span></td></tr>]], [[]], [[KENDRICK LAMAR | NOT LIKE US], [], [PGLANG/INTERSCOPE (UMG)]], [[37,946,348]], [[11%]]]","[37,946,348]","[[KENDRICK LAMAR | NOT LIKE US], [PGLANG/INTERSCOPE (UMG)]]"
2,"[[[]], [[KENDRICK LAMAR | NOT LIKE US], [], [PGLANG/INTERSCOPE (UMG)]]]",None,"[[KENDRICK LAMAR | NOT LIKE US], [PGLANG/INTERSCOPE (UMG)]]"
3,"[[\n1 ], [2 ], [[<tr><td><img class=""hits_album_chart_featured_image"" src=""https://ed.hitsdailydouble.com/inc/hits/sales/SHABOOZEY - A BAR SONG (TIPSY).png""/></td><td class=""hits_album_chart_item_top_full_details""><span class=""hits_album_chart_item_top_full_details_artist"">SHABOOZEY | A BAR SONG (TIPSY)</span><br/><span class=""hits_album_chart_item_top_dull_details_release"">EMPIRE (EMPIRE)</span></td></tr>]], [[]], [[SHABOOZEY | A BAR SONG (TIPSY)], [], [EMPIRE (EMPIRE)]], [[35,455,412]], [[-3%]]]","[35,455,412]","[[SHABOOZEY | A BAR SONG (TIPSY)], [EMPIRE (EMPIRE)]]"
4,"[[[]], [[SHABOOZEY | A BAR SONG (TIPSY)], [], [EMPIRE (EMPIRE)]]]",None,"[[SHABOOZEY | A BAR SONG (TIPSY)], [EMPIRE (EMPIRE)]]"
...,...,...,...
96,"[[\n--], [48 ], [[LUKE COMBS | AIN'T NO LOVE IN OKLAHOMA], [], [RIVER HOUSE/COLUMBIA NASHVILLE (SME)]]]",None,"[[LUKE COMBS | AIN'T NO LOVE IN OKLAHOMA], [RIVER HOUSE/COLUMBIA NASHVILLE (SME)]]"
97,"[[[<table class=""hits_album_chart_full""><tr><td class=""hits_album_chart_full_tw"">\n41 </td><td class=""hits_album_chart_full_lw"">49 </td><td class=""hits_album_chart_full_item""><span class=""hits_album_chart_item_details_full_artist"">JELLY ROLL | I AM NOT OKAY</span><br/><span class=""hits_album_chart_item_details_release"">STONEY CREEK/BMG/REPUBLIC (UMG)</span></td></tr></table>]], [\n41 ], [49 ], [[JELLY ROLL | I AM NOT OKAY], [], [STONEY CREEK/BMG/REPUBLIC (UMG)]], [[8,749,345]], [[2%]]]","[8,749,345]","[[JELLY ROLL | I AM NOT OKAY], [STONEY CREEK/BMG/REPUBLIC (UMG)]]"
98,"[[\n41 ], [49 ], [[JELLY ROLL | I AM NOT OKAY], [], [STONEY CREEK/BMG/REPUBLIC (UMG)]]]",None,"[[JELLY ROLL | I AM NOT OKAY], [STONEY CREEK/BMG/REPUBLIC (UMG)]]"
99,"[[[<table class=""hits_album_chart_full""><tr><td class=""hits_album_chart_full_tw"">\n44 </td><td class=""hits_album_chart_full_lw"">50 </td><td class=""hits_album_chart_full_item""><span class=""hits_album_chart_item_details_full_artist"">KAROL G | SI ANTES TE HUBIERA CONOCIDO</span><br/><span class=""hits_album_chart_item_details_release"">BICHOTA/INTERSCOPE (UMG)</span></td></tr></table>]], [\n44 ], [50 ], [[KAROL G | SI ANTES TE HUBIERA CONOCIDO], [], [BICHOTA/INTERSCOPE (UMG)]], [[8,257,546]], [[-1%]]]","[8,257,546]","[[KAROL G | SI ANTES TE HUBIERA CONOCIDO], [BICHOTA/INTERSCOPE (UMG)]]"


In [17]:
#creating a list of indeces that have the following variables: last week rank, current week rank, artist, song title, record labels
list_of_indeces_for_ranking_nums = []

#the indeces with the variables we want are on odd numbers for the first 5 ranked songs
for i in range(1,11,2):
    list_of_indeces_for_ranking_nums.append(i)

#the indeces with the variables we want are on even numbers all other ranks
for i in range(12,101,2):
    list_of_indeces_for_ranking_nums.append(i)

#to confirm that there are 50 indeces
print(len(list_of_indeces_for_ranking_nums))
# list_of_indeces_for_ranking_nums


50


In [18]:
# for i in list_of_indeces_for_ranking_nums:
#     print(df_temp.iloc[i,0].text.strip('\n'))

In [19]:
# """
# Older HITS Streaming Charts
# """

# lw = [] #last week's ranks
# tw = [] #this week's ranks
# artist_list = []
# song_title_list = []

# for i in list_of_indeces_for_ranking_nums:
#     #for weekly ranks
#     lw.append(df_temp.iloc[i,0][0].text.strip('\n'))
#     tw.append(df_temp.iloc[i,0][1].text.strip('\n'))

#     #for artist names
#     artist_list.append(df_temp['span'][i][0].text)

#     #for song name
#     song_title_list.append(df_temp['span'][i][1].text)


# label = []
# list_of_stream_nums = []
# change_pcnt = []

# #the indeces with the variables we want are on odd numbers for the first 5 ranked songs
# for i in range(1,11,2):
#     list_of_indeces_for_ranking_nums.append(i)
#     label.append(df_temp.iloc[i,0][5].text)
#     list_of_stream_nums.append(df_temp.iloc[i,0][6].text)
#     change_pcnt.append(df_temp.iloc[i][0][7].text)

# #the indeces with the variables we want are on even numbers all other ranks
# for i in range(11,101,2):
#     list_of_indeces_for_ranking_nums.append(i)
#     label.append(df_temp.iloc[i,0][4].text)
#     list_of_stream_nums.append(df_temp.iloc[i,0][5].text)
#     change_pcnt.append(df_temp.iloc[i][0][6].text)

In [20]:
"""
Newer HITS Streaming Charts
"""

lw = [] #last week's ranks
tw = [] #this week's ranks
artist_list = []
song_title_list = []
label = []

for i in list_of_indeces_for_ranking_nums:
    #for weekly ranks
    lw.append(df_temp.iloc[i,0][0].text.strip('\n'))
    tw.append(df_temp.iloc[i,0][1].text.strip('\n'))
    
    #for artist name
    artist = df_temp.iloc[i,2][0].text.split(" | ")[0]
    artist = artist.split(" &")[0] #gets rid of everything after " &"
    artist = artist.split(" FEAT.")[0] #gets rid of everthing after" FEAT."
    artist_list.append(artist)
    
    #for record label
    label.append(df_temp.iloc[i,2][1].text)
    
    #for song title
    song_title_list.append(df_temp.iloc[i,2][0].text.split(" | ")[1])

#finding information for the weekly streams column
list_of_stream_nums = []

#weekly streams are found on every odd index (after 1) 
for i in range(1,101,2):
    list_of_stream_nums.append(df_temp.iloc[i][1].text)

#finding information for the percent change column
change_pcnt = []

#depending on whether song is in the top 5 or not, % change is found in different locations of df_temp
for i in range(1,11,2):
    change_pcnt.append(df_temp.iloc[i][0][6].text)

for i in range(11,101,2):
    change_pcnt.append(df_temp.iloc[i][0][5].text)
change_pcnt

['11%',
 '-3%',
 '-3%',
 '--',
 '-12%',
 '-6%',
 '-4%',
 '26%',
 '0%',
 '--',
 '-3%',
 '3%',
 '-2%',
 '--',
 '-3%',
 '-3%',
 '2%',
 '1%',
 '5%',
 '15%',
 '-1%',
 '-14%',
 '7%',
 '1%',
 '--',
 '5%',
 '--',
 '-11%',
 '--',
 '-5%',
 '4%',
 '-3%',
 '--',
 '4%',
 '-5%',
 '-3%',
 '--',
 '-6%',
 '--',
 '2%',
 '1%',
 '2%',
 '-6%',
 '-2%',
 '--',
 '--',
 '1%',
 '--',
 '2%',
 '-1%']

In [21]:
#create final dataframe from HITS Daily Double
hits_streaming_df =  pd.DataFrame(list(zip(lw,tw,artist_list,song_title_list,label,
            list_of_stream_nums,change_pcnt)),
        columns=['LW', 'TW', 'Artist','Title','Label','Streams'
                ,'Change'])

#change streams from strings to integers
hits_streaming_df['Streams'] = hits_streaming_df['Streams'].str.replace(',','').astype('int')

# #get rid of apostraphes and "...?" from artist names and song titles
# df['artist'] = df['artist'].str.replace("’","").str.replace("'","").str.lower()
# df['title'] = df['title'].str.replace("’","").str.replace("'","").replace("'","").str.lower()
# df['title'] = df['title'].str.strip("…?")

#get rid of everything in parentheses as well as whitespaces
# df['title'] = df['title'].str.replace(r"\(.*\)","")
# df['title'] = df['title'].str.strip(" ")

# #get the first artist of a list if metro boomin is the lead artist
# for i in range(len(df)):
#     #print(df['artist'][i][:13])
#     if df['artist'][i][:13] == 'metro boomin,':
#         df['artist'][i] = 'metro boomin'

def convert_percent_to_float(percent_str):
    # Remove the percent sign and convert to float
    try:
        return float(percent_str.replace('%', ''))/100
    except ValueError:
        return None  # or return 0 if you prefer to handle errors that way

# Example usage
percent_strs = ['-13%', '5%', '-7.5%', '0%']
converted_floats = [convert_percent_to_float(ps) for ps in percent_strs]


hits_streaming_df['Change'] = hits_streaming_df.Change.apply(convert_percent_to_float)

hits_streaming_df['lw_streams'] = hits_streaming_df['Streams'] / (1+hits_streaming_df['Change'])

# Format the columns to not use decimals and to include commas
hits_streaming_df[['streams', 'lw_streams']] = hits_streaming_df[['Streams', 'lw_streams']].applymap(lambda x: f"{x:,.0f}")

#to make sure we have 50 songs
print(len(hits_streaming_df))

hits_streaming_df

50


,LW,TW,Artist,Title,Label,Streams,Change,lw_streams,streams
0,2,1,KENDRICK LAMAR,NOT LIKE US,PGLANG/INTERSCOPE (UMG),37946348,0.11,"34,185,899","37,946,348"
1,1,2,SHABOOZEY,A BAR SONG (TIPSY),EMPIRE (EMPIRE),35455412,-0.03,"36,551,971","35,455,412"
2,3,3,POST MALONE,I HAD SOME HELP,BIG LOUD/MERCURY/REPUBLIC (UMG),31128979,-0.03,"32,091,731","31,128,979"
3,--,4,MORGAN WALLEN,LIES LIES LIES,BIG LOUD/MERCURY/REPUBLIC (UMG),27814597,NaN,nan,"27,814,597"
4,4,5,SABRINA CARPENTER,PLEASE PLEASE PLEASE,ISLAND (UMG),27730455,-0.12,"31,511,881","27,730,455"
5,5,6,SABRINA CARPENTER,ESPRESSO,ISLAND (UMG),24892939,-0.06,"26,481,850","24,892,939"
6,6,7,TOMMY RICHMAN,MILLION DOLLAR BABY,ISO SUPREMACY/PULSE (STEM),24471626,-0.04,"25,491,277","24,471,626"
7,9,8,ZACH BRYAN,PINK SKIES,WARNER (WMG),24144253,0.26,"19,162,106","24,144,253"
8,7,9,BILLIE EILISH,BIRDS OF A FEATHER,DARKROOM/INTERSCOPE (UMG),22080759,0.00,"22,080,759","22,080,759"
9,--,10,ZACH BRYAN,28,WARNER (WMG),21464188,NaN,nan,"21,464,188"


In [22]:
hits_streaming_df.to_csv('hits_streaming.csv')

In [23]:
#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to edit google sheets from python
from df2gspread import df2gspread as d2g

In [24]:
# url = "https://docs.google.com/spreadsheets/d/1ItqY28l7EFSYr5jcFnYWHvhYyorLNBYR7RPXFZ0XjA8/edit#gid=0"

# scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)
# gc = gspread.authorize(credentials)
# spreadsheet_key = url.split("/")[-2]

# d2g.upload(revenue_df, spreadsheet_key, 'HITS Revenue', credentials = credentials,
#             row_names=False) 

# list_of_sheet_names = ['HITS Revenue','HITS Streaming','billboard']
# list_of_dfs = [revenue_df,hits_streaming_df,bb_df]

# # # Loop through both lists simultaneously using zip()
# # for sheet_name, df in zip(list_of_sheet_names, list_of_dfs):
# #     d2g.upload(df, spreadsheet_key, sheet_name, credentials = credentials,
# #             row_names=False) 

In [25]:
# Define a dictionary mapping old column names to new ones
new_column_names = {
    'Track': 'Title',
    'Sales Revenue': 'Total Sales',
    'Sales': 'Sales Revenue',
    'Streams Revenue': 'Total Streams',
    'Streams': 'Streams Revenue',
    'Streams_editted': 'Edited Streams Count'
}

revenue_df = revenue_df.rename(columns=new_column_names)
revenue_df

,LW,TW,Artist,Title,Label,Total Revenue,Total Sales,Sales Revenue,Total Streams,Streams Revenue,Edited Streams Count
0,1,1,SHABOOZEY,A BAR SONG (TIPSY),EMPIRE (EMPIRE),"$129,165","$12,740","14,000","$129,165",28388000,28388154
1,2,2,KENDRICK LAMAR,NOT LIKE US,PGLANG/INTERSCOPE (UMG),"$124,242","$3,731","4,100","$124,242",27306000,27306427
2,3,3,POST MALONE FEAT. MORGAN WALLEN,I HAD SOME HELP,BIG LOUD/MERCURY/REPUBLIC (UMG),"$121,467","$6,188","6,800","$121,467",26696000,26695852
3,4,4,JIMIN,WHO,BIGHIT/GEFFEN (UMG),"$80,799","$43,862","48,200","$80,799",17758000,17757518
4,9,5,BILLIE EILISH,BIRDS OF A FEATHER,DARKROOM/INTERSCOPE (UMG),"$93,261","$2,912","3,200","$93,261",20497000,20497308
5,7,6,CHAPPELL ROAN,"GOOD LUCK, BABE!",ISLAND (UMG),"$92,183","$2,639","2,900","$92,183",20260000,20260151
6,5,7,SABRINA CARPENTER,PLEASE PLEASE PLEASE,ISLAND (UMG),"$88,602","$2,184","2,400","$88,602",19473000,19473334
7,8,8,SABRINA CARPENTER,ESPRESSO,ISLAND (UMG),"$86,878","$3,367","3,700","$86,878",19094000,19094419
8,6,9,TOMMY RICHMAN,MILLION DOLLAR BABY,ISO SUPREMACY/PULSE (STEM),"$86,864","$2,548","2,800","$86,864",19091000,19090725
9,10,10,MORGAN WALLEN,LIES LIES LIES,BIG LOUD/MERCURY/REPUBLIC (UMG),"$76,704","$2,639","2,900","$76,704",16858000,16858205


In [26]:
revenue_df.to_csv('revenue and streaming.csv')